In [59]:
from IPython.display import Javascript, display
from ipywidgets import widgets
def run(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
tab = widgets.Button(description="Run all below")
tab.on_click(run)
display(tab)

Button(description='Run all below', style=ButtonStyle())

In [60]:
from bs4 import BeautifulSoup
import urllib.request, urllib.parse, urllib.error
import re
import time
import pandas as pd
pd.set_option('display.max_rows', 750)
pd.set_option('display.max_columns', 600)
pd.set_option('display.width', 1000)

In [61]:
#Extracting the Index
def extractindex(movie):
    element=movie.find(class_='lister-item-index unbold text-primary')
    if element is None:
        return None
    else:
        element1=element.text
        return int(element1.split('.')[0])

In [62]:
#Extracting the Genre
def extractgenre(movie):
    element=movie.find(class_='genre')
    if element is None:
        return None
    else:
        return element.text.strip().replace('\n','')

In [63]:
#Extracting the Name
def extractname(movie):
    elementhead=movie.find(class_='lister-item-header')
    if elementhead is None:
        return None
    else:
        elementhead1=elementhead.find('a').text
        return elementhead1

In [64]:
#extracting Release Year
def extractyear(movie):
    elementyear=movie.find(class_='lister-item-year text-muted unbold')
    element=re.findall(r'\((\d+)\)',elementyear.text)
    if len(element)<1:
        return None
    else:
        return int(element[0])

In [65]:
#Extracting MetaScrore
def extractscore(movie):
    temp=movie.find(class_='metascore')
    if temp is None:
        return None
    else:
        return str(temp.text.strip())

In [66]:
#Extracting Summary
def extractsummary(movie):
    temp=movie.find_all('p',{'class':'text-muted'})
    if temp is None:
        return None
    else:
        temp=temp[-1].text
        return str(temp.replace('\n','').strip())

In [67]:
#Extracting IMDB Rating
def extractrating(movie):
    elementrate=movie.find(class_='inline-block ratings-imdb-rating')
    if elementrate is None:
        return None
    else:
        elementrate1=elementrate.find('strong').text
        return float(elementrate1)

In [68]:
#Extracting Votes
def extractvote(movie):
    temp=movie.find(class_='sort-num_votes-visible')
    temp=temp.text.strip().replace('\n','')
    vote=re.findall(r'Vote[s:]+([0-9,]+)',temp)
    if len(vote)<1:
        return None
    else:
        return vote[0]

In [69]:
#Extracting Director and Starrers
def directorstarrer(movie):
    one=[]
    temp=''
    elementdirectorstarrer=movie.find_all('p',{'class':''})
    for i in elementdirectorstarrer:
        if 'Director' in i.text:
            temp=i.text
    temp=temp.replace('\n','').strip()   
    #Using RE to get starrers and directors
    director=re.findall(r'^Director[s:]*([a-zA-Z0-9 ,.]*)',temp)
    if len(director)<1:
        one.append(None)
    else:
        one.append(director[0])
    star=re.findall(r'Star[s:]+([a-zA-Z0-9 ,.]*)',temp)
    if len(star)<1:
        one.append(None)
    else:
        one.append(star[0])
    return one

In [70]:
#Fetching URL
def fetchurl(url=''):
    if url=='':
        url='https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&ref_=adv_prv'
    user_agent={}
    user_agent['Mozila'] = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    user_agent['Android'] = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:68.0) Gecko/20100101 Firefox/68.0'
    headers={'User-Agent':user_agent['Mozila'],'Accept-Language': 'en-US,en;q=0.8'}
    urlHandle=urllib.request.Request(url,None,headers)
    urlHandle=urllib.request.urlopen(urlHandle)
    content=urlHandle.read()
    return BeautifulSoup(content,'html.parser')

In [71]:
#Finding Next URL
def findnexturl(s):
    link=s.find(class_='lister-page-next next-page')
    if link is None:
        return None
    else:
        link=link.get('href')
        link='https://www.imdb.com'+link
    return link

In [72]:
#Extracting Gross
def extractgross(movie):
    temp=movie.find(class_='sort-num_votes-visible')
    temp=temp.text.strip().replace('\n','')
    gross=re.findall(r'Gross:([\$0-9.,A-Z]+)',temp)
    if len(gross)<1:
        return None
    else:
        return gross[0]

In [73]:
#DataFrame
def df():
    data={}
    data['Name']=name
    data['Year of Release']=year
    data['Genre']=genre
    data['IMDB Rating']=IMDB_rating
    data['Metascore']=metascore
    data['Director(s)']=directors
    data['Star(s)']=stars
    data['Summary']=summary
    data['Votes']=votes
    data['Gross Earning']=grosses
    df = pd.DataFrame(data=data,index=index)
    df.to_csv('IMDB.csv')

In [84]:
index=[]
name=[]
year=[]
genre=[]
IMDB_rating=[]
metascore=[]
summary=[]
directors=[]
stars=[]
votes=[]
grosses=[]

def main():
    waittime=input('How long should the user wait in seconds? (default: 10): ')
    if waittime=='':
        waittime=10
    else:
        waittime=int(waittime)
    print('Getting you. Please wait!')
    s=fetchurl()
    for i in range(1,6):
        movies=s.find_all(class_='lister-item-content')
        print('Getting data')
        for movie in movies:
            #Procuring all info
            index.append(extractindex(movie))
            name.append(extractname(movie))
            year.append(extractyear(movie))
            genre.append(extractgenre(movie))
            IMDB_rating.append(extractrating(movie))
            metascore.append(extractscore(movie))
            summary.append(extractsummary(movie))
            directornstar=directorstarrer(movie)
            directors.append(directornstar[0])
            stars.append(directornstar[1])
            votes.append(extractvote(movie))
            grosses.append(extractgross(movie))
        next_link=findnexturl(s)
        if next_link is None:
            print('Exiting')
            break;
        print('Waiting ',waittime,' seconds')
        time.sleep(waittime)
        s=fetchurl(next_link)
    df()

In [85]:
main()

How long should the user wait in seconds? (default: 10): 1
Getting you. Please wait!
Getting data
Waiting  1  seconds
Getting data
Waiting  1  seconds
Getting data
Waiting  1  seconds
Getting data
Waiting  1  seconds
Getting data
Exiting
